In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset,concatenate_datasets
import transformers
from sklearn.metrics import confusion_matrix

from transformers import AutoTokenizer, DataCollatorWithPadding,RobertaForSequenceClassification,BertForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer

from src.utils.myutils import *


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

CS_DATA_PATH = PATH + '/data/CS/raw'
EN_DATA_PATH = PATH + '/data/EN/raw'

model_checkpoint = 'ufal/robeczech-base'
BATCH_SIZE = 32
transformers.utils.logging.set_verbosity_error()

In [3]:
train = load_dataset('csv',data_files=CS_DATA_PATH+"/WIKI2/wiki2.csv")['train']
babe_cs = load_dataset('csv',data_files=CS_DATA_PATH + '/BABE/SG2.csv')['train']

Using custom data configuration default-372cac647a45880f
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-372cac647a45880f/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Using custom data configuration default-41acc90be2294f89
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-41acc90be2294f89/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [60]:
train = load_dataset('csv',data_files=CS_DATA_PATH+"/WikiBias/wikibias.csv")['train']

Using custom data configuration default-ee602d71f2907259


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/horyctom/.cache/huggingface/datasets/csv/default-ee602d71f2907259/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


In [2]:
training_args = TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    #per_device_eval_batch_size=BATCH_SIZE,
    logging_steps=100,
    disable_tqdm = False,
    warmup_steps=50,
    save_total_limit=2,
    #load_best_model_at_end=True,
    #evaluation_strategy="steps",
    #metric_for_best_model = 'f1',
    weight_decay=0.1,
    output_dir = './',
    learning_rate=4e-5)

## BEST PARAMS

In [20]:
czert = BertForSequenceClassification.from_pretrained('UWB-AIR/Czert-B-base-cased',num_labels=2)
robeczech = RobertaForSequenceClassification.from_pretrained('ufal/robeczech-base',num_labels=2);
multilingual = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=2)
fernet_c5 = BertForSequenceClassification.from_pretrained('fav-kky/FERNET-C5',num_labels=2)
fernet_news = RobertaForSequenceClassification.from_pretrained('fav-kky/FERNET-News',num_labels=2)
slavic_bert = BertForSequenceClassification.from_pretrained('DeepPavlov/bert-base-bg-cs-pl-ru-cased',num_labels=2)
xlm_roberta_large = RobertaForSequenceClassification.from_pretrained('xlm-roberta-large',num_labels=2)


models = [czert,robeczech,multilingual,fernet_c5,fernet_news,slavic_bert,xlm_roberta_large]

loading configuration file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/1109a10013d6f221417ff52a1198bdc115db12e68d957a317c3cc463b17f5d43.28e3a761a3c5e32166573d4cc31b0a71ae1b56a98381799f8ff5332f26600fd7
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/pytorch_model.bin from cache at /home/horyctom/.cache/huggingface/transformers/6e

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

storing https://huggingface.co/DeepPavlov/bert-base-bg-cs-pl-ru-cased/resolve/main/config.json in cache at /home/horyctom/.cache/huggingface/transformers/969927d8dc3ef116510e05257d0982836522039e0a62148f115b2cd116f6dafb.e8f15c5aad2f4653e46ceeba0bb32c02a629d106a902c964bce60523d290ac8f
creating metadata file for /home/horyctom/.cache/huggingface/transformers/969927d8dc3ef116510e05257d0982836522039e0a62148f115b2cd116f6dafb.e8f15c5aad2f4653e46ceeba0bb32c02a629d106a902c964bce60523d290ac8f
loading configuration file https://huggingface.co/DeepPavlov/bert-base-bg-cs-pl-ru-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/969927d8dc3ef116510e05257d0982836522039e0a62148f115b2cd116f6dafb.e8f15c5aad2f4653e46ceeba0bb32c02a629d106a902c964bce60523d290ac8f
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dro

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

storing https://huggingface.co/DeepPavlov/bert-base-bg-cs-pl-ru-cased/resolve/main/pytorch_model.bin in cache at /home/horyctom/.cache/huggingface/transformers/be513d89ab94907263845963afdaf9e88c24707b010ad7dbaf122ebc9a9d221c.99a2af41d33109734fbe3b358898456638a02ac3964600e9b37dd048f06d32c2
creating metadata file for /home/horyctom/.cache/huggingface/transformers/be513d89ab94907263845963afdaf9e88c24707b010ad7dbaf122ebc9a9d221c.99a2af41d33109734fbe3b358898456638a02ac3964600e9b37dd048f06d32c2
loading weights file https://huggingface.co/DeepPavlov/bert-base-bg-cs-pl-ru-cased/resolve/main/pytorch_model.bin from cache at /home/horyctom/.cache/huggingface/transformers/be513d89ab94907263845963afdaf9e88c24707b010ad7dbaf122ebc9a9d221c.99a2af41d33109734fbe3b358898456638a02ac3964600e9b37dd048f06d32c2
Some weights of the model checkpoint at DeepPavlov/bert-base-bg-cs-pl-ru-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias'

Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

storing https://huggingface.co/xlm-roberta-large/resolve/main/config.json in cache at /home/horyctom/.cache/huggingface/transformers/4d7a1550c9ab8701667bc307a1213c040fcc06dc87a5e4994e72aecc0d7e0337.842c7737719967568f4691849854475018d6cf7ce21f52576bb6e0d10091bd3c
creating metadata file for /home/horyctom/.cache/huggingface/transformers/4d7a1550c9ab8701667bc307a1213c040fcc06dc87a5e4994e72aecc0d7e0337.842c7737719967568f4691849854475018d6cf7ce21f52576bb6e0d10091bd3c
loading configuration file https://huggingface.co/xlm-roberta-large/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/4d7a1550c9ab8701667bc307a1213c040fcc06dc87a5e4994e72aecc0d7e0337.842c7737719967568f4691849854475018d6cf7ce21f52576bb6e0d10091bd3c
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Model config RobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "atte

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

storing https://huggingface.co/xlm-roberta-large/resolve/main/pytorch_model.bin in cache at /home/horyctom/.cache/huggingface/transformers/4b3ca85a63804fb7cd317765d9de19ce6208ee0fc9691b209384ee7cfd9cb3b9.64b4693d874c772310b8acda9a1193cfade77d56795a9b488e612f198b68f6f7
creating metadata file for /home/horyctom/.cache/huggingface/transformers/4b3ca85a63804fb7cd317765d9de19ce6208ee0fc9691b209384ee7cfd9cb3b9.64b4693d874c772310b8acda9a1193cfade77d56795a9b488e612f198b68f6f7
loading weights file https://huggingface.co/xlm-roberta-large/resolve/main/pytorch_model.bin from cache at /home/horyctom/.cache/huggingface/transformers/4b3ca85a63804fb7cd317765d9de19ce6208ee0fc9691b209384ee7cfd9cb3b9.64b4693d874c772310b8acda9a1193cfade77d56795a9b488e612f198b68f6f7
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.w

In [25]:
babe_eval = babe_cs.train_test_split(0.15)
babe_eval

Loading cached split indices for dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-41acc90be2294f89/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-04f82fc556c52f84.arrow and /home/horyctom/.cache/huggingface/datasets/csv/default-41acc90be2294f89/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-ff79a5038a37cc62.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 3122
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 551
    })
})

In [26]:
scores_on_babe = {}
for model in models:
    
    model.to(device)
    tokenizer = AutoTokenizer.from_pretrained(model.name_or_path, use_fast=False,padding=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    train_tok = preprocess_data(babe_eval['train'],tokenizer,'sentence')
    test_tok = preprocess_data(babe_eval['test'],tokenizer,'sentence')
    
    print("Fitting ", model.name_or_path,"...")
    trainer = Trainer(model,training_args,train_dataset=train_tok,data_collator=data_collator,
                          tokenizer=tokenizer)
    trainer.train()
    print("Done.")
    test_dataloader = DataLoader(test_tok, batch_size=BATCH_SIZE, collate_fn=data_collator)
    scores_on_babe[model.name_or_path] = compute_metrics(model,device,test_dataloader)

loading configuration file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/1109a10013d6f221417ff52a1198bdc115db12e68d957a317c3cc463b17f5d43.28e3a761a3c5e32166573d4cc31b0a71ae1b56a98381799f8ff5332f26600fd7
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/vocab.txt f

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

***** Running training *****
  Num examples = 3122
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 294


Fitting  UWB-AIR/Czert-B-base-cased ...


...

***** Running training *****
  Num examples = 3122
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 294


Fitting  xlm-roberta-large ...


Step,Training Loss
100,0.690100
200,0.529900




Training completed. Do not forget to share your model on huggingface.co/models =)




Done.


In [27]:
scores_on_babe

{'UWB-AIR/Czert-B-base-cased': {'f1': 0.7767695099818511},
 'ufal/robeczech-base': {'f1': 0.7749546279491834},
 'bert-base-multilingual-cased': {'f1': 0.7277676950998184},
 'fav-kky/FERNET-C5': {'f1': 0.7749546279491834},
 'fav-kky/FERNET-News': {'f1': 0.7931034482758621},
 'DeepPavlov/bert-base-bg-cs-pl-ru-cased': {'f1': 0.7912885662431943},
 'xlm-roberta-large': {'f1': 0.7822141560798548}}

## Experiment on the best one

In [36]:
from sklearn.model_selection import StratifiedKFold

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [37]:
scores = []
model_checkpoint = 'fav-kky/FERNET-News'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False,padding=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
babe_tok = preprocess_data(babe_cs,tokenizer,'sentence')

for train_index, val_index in skfold.split(babe_tok['input_ids'],babe_tok['label']):
    
    token_train = Dataset.from_dict(babe_tok[train_index])
    token_valid = Dataset.from_dict(babe_tok[val_index])
    
    #augmentation
    #aug_tok = preprocess_data(train,tokenizer,'sentence')
    #token_train = concatenate_datasets([aug_tok,token_train])
    
    model = RobertaForSequenceClassification.from_pretrained(model_checkpoint,num_labels=2);
    model.to(device)
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,
                      tokenizer=tokenizer)
    trainer.train()
    
    #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    scores.append(compute_metrics(model,device,eval_dataloader))


loading configuration file https://huggingface.co/fav-kky/FERNET-News/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/f00d2cfe5a7793b975db2f5aaf09909f4ac88802bc3d6471179dd8411c320e16.a971189d67e3fb9209861350e9064bdb282be80f780f9af15ec6180b612da726
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50000
}

loading file https://huggingface.co/fav-kky/FERNET-News/resolve/main/voca

Step,Training Loss
100,0.677700
200,0.533200
300,0.381400
400,0.254300




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/fav-kky/FERNET-News/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/f00d2cfe5a7793b975db2f5aaf09909f4ac88802bc3d6471179dd8411c320e16.a971189d67e3fb9209861350e9064bdb282be80f780f9af15ec6180b612da726
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "ge

Step,Training Loss
100,0.671400
200,0.495500
300,0.332600
400,0.211100




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/fav-kky/FERNET-News/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/f00d2cfe5a7793b975db2f5aaf09909f4ac88802bc3d6471179dd8411c320e16.a971189d67e3fb9209861350e9064bdb282be80f780f9af15ec6180b612da726
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "ge

...



Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/fav-kky/FERNET-News/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/f00d2cfe5a7793b975db2f5aaf09909f4ac88802bc3d6471179dd8411c320e16.a971189d67e3fb9209861350e9064bdb282be80f780f9af15ec6180b612da726
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "ge

Step,Training Loss
100,0.714100
200,0.706500
300,0.701000
400,0.697000




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/fav-kky/FERNET-News/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/f00d2cfe5a7793b975db2f5aaf09909f4ac88802bc3d6471179dd8411c320e16.a971189d67e3fb9209861350e9064bdb282be80f780f9af15ec6180b612da726
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "ge

Step,Training Loss
100,0.714100
200,0.591400
300,0.498400
400,0.376200




Training completed. Do not forget to share your model on huggingface.co/models =)




In [39]:
score = 0
for d in scores:
    score += d['f1']
score/5

0.7217097629242433

In [38]:
scores

[{'f1': 0.7863945578231294},
 {'f1': 0.7619047619047619},
 {'f1': 0.7768707482993197},
 {'f1': 0.5068119891008175},
 {'f1': 0.776566757493188}]

### Pretrain multilingual on WNC en

In [4]:
wnc = load_dataset('csv',data_files=PATH+"/data/EN/processed/WNC/wnc.csv")['train']

Using custom data configuration default-8a96bc76b9b8d191
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-8a96bc76b9b8d191/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [6]:
model = RobertaForSequenceClassification.from_pretrained('xlm-roberta-large',num_labels=2)
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large', use_fast=False,padding=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
wnc_tok = preprocess_data(wnc,tokenizer,'sentence')

Loading cached processed dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-8a96bc76b9b8d191/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-13c7b879213648d9.arrow


In [7]:
model.to(device)
trainer = Trainer(model,training_args,train_dataset=wnc_tok,data_collator=data_collator,
                      tokenizer=tokenizer)
trainer.train()

***** Running training *****
  Num examples = 362990
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 34032


Step,Training Loss


RuntimeError: CUDA out of memory. Tried to allocate 42.00 MiB (GPU 0; 31.75 GiB total capacity; 30.29 GiB already allocated; 10.00 MiB free; 30.89 GiB reserved in total by PyTorch)